# Installing necessary libraries

In [1]:
!pip install PyPDF2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 2.1 MB/s eta 0:00:00


In [2]:
!pip install sentence-transformers==2.2.2 llama-cpp-python langchain openai pypdf chromadb tiktoken InstructorEmbedding ragas arxiv pymupdf
!pip install -q -U tqdm datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 857.9 kB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 MB 11.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 973.5/973.5 kB 73.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.6/320.6 kB 34.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.4/290.4 kB 39.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 526.8/526.8 kB 53.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 71.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 13.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.5/3.5 MB 57.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4

# Importing Nessesary Libraries

In [3]:
from langchain.document_loaders import PyPDFLoader
from langchain.vectorstores import Chroma
from langchain.embeddings import HuggingFaceInstructEmbeddings
from langchain.embeddings import HuggingFaceBgeEmbeddings
from langchain.embeddings import HuggingFaceEmbeddings
from chromadb.config import Settings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains import RetrievalQA

from huggingface_hub import hf_hub_download
from langchain.llms import LlamaCpp
from transformers import AutoModelForCausalLM, AutoTokenizer, LlamaForCausalLM, LlamaTokenizer, BitsAndBytesConfig
from langchain.prompts import PromptTemplate

from ragas import evaluate

import os
import openai
from getpass import getpass

from langchain.output_parsers import ResponseSchema
from langchain.output_parsers import StructuredOutputParser
from tqdm import tqdm
# import pandas as pd
from ragas.metrics.critique import harmfulness
from ragas import evaluate
from langchain.document_loaders import ArxivLoader
from datasets import Dataset
import ragas
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.prompts import ChatPromptTemplate
from tqdm import tqdm

# Defining Constants

In [4]:
# model used for creating embeddings
EMBEDDING_MODEL_NAME = "hkunlp/instructor-large"

MODEL_ID = "TheBloke/Llama-2-7b-Chat-GGUF"
MODEL_BASENAME = "llama-2-7b-chat.Q4_K_M.gguf"
MODELS_PATH = "/content/models"

CONTEXT_WINDOW_SIZE = 4096
MAX_NEW_TOKENS = CONTEXT_WINDOW_SIZE
N_GPU_LAYERS = 100
N_BATCH = 512

# Downloading LL Models and Embedding Models

In [5]:
# function for creating embeddings
def get_embeddings(device_type="cuda"):
    if "instructor" in EMBEDDING_MODEL_NAME:
        return HuggingFaceInstructEmbeddings(
            model_name=EMBEDDING_MODEL_NAME,
            model_kwargs={"device": device_type},
            embed_instruction="Represent the document for retrieval:",
            query_instruction="Represent the question for retrieving supporting documents:",
        )

    elif "bge" in EMBEDDING_MODEL_NAME:
        return HuggingFaceBgeEmbeddings(
            model_name=EMBEDDING_MODEL_NAME,
            model_kwargs={"device": device_type},
            query_instruction="Represent this sentence for searching relevant passages:",
        )

    else:
        return HuggingFaceEmbeddings(
            model_name=EMBEDDING_MODEL_NAME,
            model_kwargs={"device": device_type},
        )

In [6]:
embeddings = get_embeddings()

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


.gitattributes:   0%|          | 0.00/1.48k [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/270 [00:00<?, ?B/s]

2_Dense/config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/3.15M [00:00<?, ?B/s]

...

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/2.41k [00:00<?, ?B/s]

modules.json:   0%|          | 0.00/461 [00:00<?, ?B/s]

load INSTRUCTOR_Transformer
max_seq_length  512


In [7]:
# downloading large language model
model_path = hf_hub_download(
            repo_id=MODEL_ID,
            filename=MODEL_BASENAME,
            resume_download=True,
            cache_dir=MODELS_PATH,
        )

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


llama-2-7b-chat.Q4_K_M.gguf:   0%|          | 0.00/4.08G [00:00<?, ?B/s]

In [8]:
kwargs = {
            "model_path": model_path,
            "n_ctx": CONTEXT_WINDOW_SIZE,
            "max_tokens": MAX_NEW_TOKENS,
            "n_batch": N_BATCH,  # set this based on your GPU & CPU RAM
            "n_gpu_layers" : N_GPU_LAYERS
        }

In [9]:
# initializing LlamaCpp wrapper to work with large language model
llm = LlamaCpp(**kwargs)

llama_model_loader: loaded meta data with 19 key-value pairs and 291 tensors from /content/models/models--TheBloke--Llama-2-7b-Chat-GGUF/snapshots/191239b3e26b2882fb562ffccdd1cf0f65402adb/llama-2-7b-chat.Q4_K_M.gguf (version GGUF V2)
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = LLaMA v2
llama_model_loader: - kv   2:                       llama.context_length u32              = 4096
llama_model_loader: - kv   3:                     llama.embedding_length u32              = 4096
llama_model_loader: - kv   4:                          llama.block_count u32              = 32
llama_model_loader: - kv   5:                  llama.feed_forward_length u32              = 11008
llama_model_loader: - kv   6:                 llama.rope.dimension_count u32      

In [10]:
llm

LlamaCpp(client=<llama_cpp.llama.Llama object at 0x7a06edac7fd0>, model_path='/content/models/models--TheBloke--Llama-2-7b-Chat-GGUF/snapshots/191239b3e26b2882fb562ffccdd1cf0f65402adb/llama-2-7b-chat.Q4_K_M.gguf', n_ctx=4096, n_batch=512, n_gpu_layers=100, max_tokens=4096)

# Question answering chain

In [11]:
template = """Use the provided context to respond to the question posed at the
end.
If you're unsure of the answer, please feel free to acknowledge that you don't
know rather than attempting to provide a fabricated response.
Please provide a brief and concise response.
{context}
Question: {question}
Helpful Answer:"""
QA_CHAIN_PROMPT = PromptTemplate.from_template(template)
QA_CHAIN_PROMPT

PromptTemplate(input_variables=['context', 'question'], template="Use the provided context to respond to the question posed at the\nend.\nIf you're unsure of the answer, please feel free to acknowledge that you don't\nknow rather than attempting to provide a fabricated response.\nPlease provide a brief and concise response.\n{context}\nQuestion: {question}\nHelpful Answer:")

## Dataset Creation For Evaluation

## OpenAI API key setup

In [12]:
openai.api_key = getpass("Enter Openai Key")
os.environ["OPENAI_API_KEY"] = openai.api_key

Enter Openai Key··········


## Downloading Documents And COnverting into Chunks

In [13]:
base_docs = ArxivLoader(query="Retrieval Augmented Generation", load_max_docs=5).load()
len(base_docs)

5

In [14]:
for doc in base_docs:
  print(doc.metadata)

{'Published': '2022-02-13', 'Title': 'A Survey on Retrieval-Augmented Text Generation', 'Authors': 'Huayang Li, Yixuan Su, Deng Cai, Yan Wang, Lemao Liu', 'Summary': 'Recently, retrieval-augmented text generation attracted increasing attention\nof the computational linguistics community. Compared with conventional\ngeneration models, retrieval-augmented text generation has remarkable\nadvantages and particularly has achieved state-of-the-art performance in many\nNLP tasks. This paper aims to conduct a survey about retrieval-augmented text\ngeneration. It firstly highlights the generic paradigm of retrieval-augmented\ngeneration, and then it reviews notable approaches according to different tasks\nincluding dialogue response generation, machine translation, and other\ngeneration tasks. Finally, it points out some important directions on top of\nrecent methods to facilitate future research.'}
{'Published': '2024-05-12', 'Title': 'DuetRAG: Collaborative Retrieval-Augmented Generation', 'A

In [15]:
base_docs[0].metadata

{'Published': '2022-02-13',
 'Title': 'A Survey on Retrieval-Augmented Text Generation',
 'Authors': 'Huayang Li, Yixuan Su, Deng Cai, Yan Wang, Lemao Liu',
 'Summary': 'Recently, retrieval-augmented text generation attracted increasing attention\nof the computational linguistics community. Compared with conventional\ngeneration models, retrieval-augmented text generation has remarkable\nadvantages and particularly has achieved state-of-the-art performance in many\nNLP tasks. This paper aims to conduct a survey about retrieval-augmented text\ngeneration. It firstly highlights the generic paradigm of retrieval-augmented\ngeneration, and then it reviews notable approaches according to different tasks\nincluding dialogue response generation, machine translation, and other\ngeneration tasks. Finally, it points out some important directions on top of\nrecent methods to facilitate future research.'}

In [16]:
# splitting resource documents into small chunks
text_splitter = RecursiveCharacterTextSplitter(
chunk_size = 1500,
chunk_overlap = 150,
separators=['. ']
)
chunks = text_splitter.split_documents(base_docs)
len(chunks)

168

In [17]:
len(chunks)

168

In [18]:
print(max([len(chunk.page_content) for chunk in chunks]))

2274


## Creating Vector Database

In [19]:
PERSIST_DIRECTORY_NEW = "/content/DB_NEW"

# Creating Vector database
vectordb = Chroma.from_documents(
documents=chunks,
embedding=embeddings,
persist_directory=PERSIST_DIRECTORY_NEW
)

In [20]:
print(chunks[0])

page_content='A Survey on Retrieval-Augmented Text Generation\nHuayang Li♥,∗\nYixuan Su♠,∗\nDeng Cai♦,∗\nYan Wang♣,∗\nLemao Liu♣,∗\n♥Nara Institute of Science and Technology\n♠University of Cambridge\n♦The Chinese University of Hong Kong\n♣Tencent AI Lab\nli.huayang.lh6@is.naist.jp, ys484@cam.ac.uk\nthisisjcykcd@gmail.com, brandenwang@tencent.com\nlemaoliu@gmail.com\nAbstract\nRecently, retrieval-augmented text generation\nattracted increasing attention of the compu-\ntational linguistics community.\nCompared\nwith conventional generation models, retrieval-\naugmented text generation has remarkable ad-\nvantages and particularly has achieved state-of-\nthe-art performance in many NLP tasks. This\npaper aims to conduct a survey about retrieval-\naugmented text generation. It ﬁrstly highlights\nthe generic paradigm of retrieval-augmented\ngeneration, and then it reviews notable ap-\nproaches according to different tasks including\ndialogue response generation, machine trans-\nlation, and

In [21]:
qa_chain = RetrievalQA.from_chain_type(
llm,
retriever=vectordb.as_retriever(),
return_source_documents=True,
chain_type_kwargs={"prompt": QA_CHAIN_PROMPT}
)

In [22]:
base_retriever = vectordb.as_retriever(search_kwargs={"k" : 2})

In [23]:
relevant_docs = base_retriever.get_relevant_documents("What is Retrieval Augmented Generation?")

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


In [24]:
len(relevant_docs)

2

In [25]:
!pip install arxiv requests

In [47]:
base_docs[0].metadata

{'Published': '2022-02-13',
 'Title': 'A Survey on Retrieval-Augmented Text Generation',
 'Authors': 'Huayang Li, Yixuan Su, Deng Cai, Yan Wang, Lemao Liu',
 'Summary': 'Recently, retrieval-augmented text generation attracted increasing attention\nof the computational linguistics community. Compared with conventional\ngeneration models, retrieval-augmented text generation has remarkable\nadvantages and particularly has achieved state-of-the-art performance in many\nNLP tasks. This paper aims to conduct a survey about retrieval-augmented text\ngeneration. It firstly highlights the generic paradigm of retrieval-augmented\ngeneration, and then it reviews notable approaches according to different tasks\nincluding dialogue response generation, machine translation, and other\ngeneration tasks. Finally, it points out some important directions on top of\nrecent methods to facilitate future research.'}

In [48]:
# import requests
# import PyPDF2

# # Extract the URLs from the base_docs list
# urls = [doc['metadata']['pdf'] for doc in base_docs]

# # Download the PDF files
# for i, url in enumerate(urls):
#     response = requests.get(url)
#     with open(f"paper_{i}.pdf", "wb") as f:
#         f.write(response.content)

# # Save the PDF files using PyPDF2
# for i, url in enumerate(urls):
#     with open(f"paper_{i}.pdf", "rb") as f:
#         pdf_file = PyPDF2.PdfFileReader(f)
#         with open(f"paper_{i}_saved.pdf", "wb") as f:
#             pdf_writer = PyPDF2.PdfFileWriter()
#             for page in range(pdf_file.getNumPages()):
#                 pdf_writer.addPage(pdf_file.getPage(page))
#             pdf_writer.write(f)

# # Clean up the temporary files
# for i in range(len(urls)):
#     try:
#         os.remove(f"paper_{i}.pdf")
#     except OSError:
#         pass

## Creating QA template

In [27]:
from langchain.output_parsers import ResponseSchema
from langchain.output_parsers import StructuredOutputParser

question_schema = ResponseSchema(
    name="question",
    description="a question about the context."
)

question_response_schemas = [
    question_schema,
]

In [28]:
question_output_parser = StructuredOutputParser.from_response_schemas(question_response_schemas)
format_instructions = question_output_parser.get_format_instructions()

In [29]:
print(format_instructions)

The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":

```json
{
	"question": string  // a question about the context.
}
```


In [30]:
question_schema = ResponseSchema(
    name="question",
    description="a question about the context."
)

question_response_schemas = [
    question_schema,
]

In [31]:
template = """Answer the question based only on the following context. If you cannot answer the question with the context, please respond with 'I don't know':

### CONTEXT
{context}

### QUESTION
Question: {question}
"""

prompt = ChatPromptTemplate.from_template(template)

## QA chain for gpt-3.5

In [32]:
from langchain.chat_models import ChatOpenAI

In [33]:
question_generation_llm = ChatOpenAI(model="gpt-3.5-turbo-16k")

bare_prompt_template = "{content}"
bare_template = ChatPromptTemplate.from_template(template=bare_prompt_template)

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


In [34]:
from langchain.prompts import ChatPromptTemplate

template = """Answer the question based only on the following context. If you cannot answer the question with the context, please respond with 'I don't know':

### CONTEXT
{context}

### QUESTION
Question: {question}
"""

prompt = ChatPromptTemplate.from_template(template)

In [35]:
question_generation_llm = ChatOpenAI(model="gpt-3.5-turbo-16k")

bare_prompt_template = "{content}"
bare_template = ChatPromptTemplate.from_template(template=bare_prompt_template)

In [36]:
from langchain.prompts import ChatPromptTemplate

qa_template = """\
You are a University Professor creating a test for advanced students. For each context, create a question that is specific to the context. Avoid creating generic or general questions.

question: a question about the context.

Format the output as JSON with the following keys:
question

context: {context}
"""

prompt_template = ChatPromptTemplate.from_template(template=qa_template)

messages = prompt_template.format_messages(
    context=chunks[0],
    format_instructions=format_instructions
)

question_generation_chain = bare_template | question_generation_llm

response = question_generation_chain.invoke({"content" : messages})
output_dict = question_output_parser.parse(response.content)

RateLimitError: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}

In [ ]:
for k, v in output_dict.items():
  print(k)
  print(v)

In [ ]:
qac_triples = []

for text in tqdm(chunks[:10]):
  messages = prompt_template.format_messages(
      context=text,
      format_instructions=format_instructions
  )
  response = question_generation_chain.invoke({"content" : messages})
  try:
    output_dict = question_output_parser.parse(response.content)
  except Exception as e:
    continue
  output_dict["context"] = text
  qac_triples.append(output_dict)

100%|██████████| 10/10 [02:47<00:00, 16.75s/it]


In [ ]:
len(qac_triples)

10

In [ ]:
answer_generation_llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)

answer_schema = ResponseSchema(
    name="answer",
    description="an answer to the question"
)

answer_response_schemas = [
    answer_schema,
]

answer_output_parser = StructuredOutputParser.from_response_schemas(answer_response_schemas)
format_instructions = answer_output_parser.get_format_instructions()

qa_template = """\
You are a University Professor creating a test for advanced students. For each question and context, create an answer.

answer: a answer about the context.

Format the output as JSON with the following keys:
answer

question: {question}
context: {context}
"""

prompt_template = ChatPromptTemplate.from_template(template=qa_template)

messages = prompt_template.format_messages(
    context=qac_triples[0]["context"],
    question=qac_triples[0]["question"],
    format_instructions=format_instructions
)

answer_generation_chain = bare_template | answer_generation_llm

response = answer_generation_chain.invoke({"content" : messages})
output_dict = answer_output_parser.parse(response.content)

In [ ]:
for k, v in output_dict.items():
  print(k)
  print(v)

answer
The main focus of this paper is to conduct a survey on retrieval-augmented text generation, highlighting its advantages and achievements in various NLP tasks.
question
What is the main focus of this paper?
context
{'page_content': 'A Survey on Retrieval-Augmented Text Generation\nHuayang Li♥,∗\nYixuan Su♠,∗\nDeng Cai♦,∗\nYan Wang♣,∗\nLemao Liu♣,∗\n♥Nara Institute of Science and Technology\n♠University of Cambridge\n♦The Chinese University of Hong Kong\n♣Tencent AI Lab\nli.huayang.lh6@is.naist.jp, ys484@cam.ac.uk\nthisisjcykcd@gmail.com, brandenwang@tencent.com\nlemaoliu@gmail.com\nAbstract\nRecently, retrieval-augmented text generation\nattracted increasing attention of the compu-\ntational linguistics community.\nCompared\nwith conventional generation models, retrieval-\naugmented text generation has remarkable ad-\nvantages and particularly has achieved state-of-\nthe-art performance in many NLP tasks. This\npaper aims to conduct a survey about retrieval-\naugmented text gener

In [ ]:
for triple in tqdm(qac_triples):
  messages = prompt_template.format_messages(
      context=triple["context"],
      question=triple["question"],
      format_instructions=format_instructions
  )
  response = answer_generation_chain.invoke({"content" : messages})
  try:
    output_dict = answer_output_parser.parse(response.content)
  except Exception as e:
    continue
  triple["answer"] = output_dict["answer"]

100%|██████████| 10/10 [02:45<00:00, 16.60s/it]


## Initial Evaluation Dataset

In [ ]:
triple

{'question': 'What are the advantages of retrieval-augmented text generation compared to conventional generation models?',
 'context': Document(page_content='. The\nretrieval-based models (Ji et al., 2014; Hu et al.,\n2014) directly copy an existing response from cu-\nrated dialogue corpora (i.e., the retrieval pool)\nwhen receiving a response request. The retrieved\nresponses are often informative and grammatical\nas they are collected from real-world conversa-\ntions and possibly post-edited by a human. How-\never, such systems perform poorly when a given\ndialogue history is substantially different from\nthose in the retrieval pool. On the other hand,\nthe generation-based models (Shang et al., 2015;\nVinyals and Le, 2015; Li et al., 2016a) generate\na new utterance from scratch. Those generation-\nbased models have better generalization capacity\nwhen handling unseen dialogue contexts. Never-\ntheless, the generated utterances are inclined to be\ndull and non-informative (e.g., “I 

In [ ]:
import pandas as pd

In [ ]:
ground_truth_qac_set = pd.DataFrame(qac_triples)
ground_truth_qac_set["context"] = ground_truth_qac_set["context"].map(lambda x: str(x.page_content))
ground_truth_qac_set = ground_truth_qac_set.rename(columns={"answer" : "ground_truth"})


eval_dataset = Dataset.from_pandas(ground_truth_qac_set)

In [ ]:
eval_dataset

Dataset({
    features: ['question', 'context', 'ground_truth'],
    num_rows: 10
})

In [ ]:
eval_dataset["question"]

['What is the main focus of this paper?',
 'What are the advantages of retrieval-augmented text generation compared to generation-based counterpart?',
 'What are some applications of retrieval-augmented generation in other generation tasks?',
 'What is the main idea behind retrieval-augmented generation?',
 'What are the three kinds of sources from which the retrieval memory can be retrieved?',
 'What is the main limitation of using supervised datasets for retrieval in machine translation?',
 'What are some methods used for similarity measurement in supervised corpus retrieval?',
 'What is one method of integrating the retrieved external memory in generation?',
 'What are the advantages of retrieval-augmented text generation compared to conventional generation models?']

In [ ]:
[str(docs.page_content) for docs in vectordb.as_retriever().get_relevant_documents(eval_dataset[0]["context"])][0]

'A Survey on Retrieval-Augmented Text Generation\nHuayang Li♥,∗\nYixuan Su♠,∗\nDeng Cai♦,∗\nYan Wang♣,∗\nLemao Liu♣,∗\n♥Nara Institute of Science and Technology\n♠University of Cambridge\n♦The Chinese University of Hong Kong\n♣Tencent AI Lab\nli.huayang.lh6@is.naist.jp, ys484@cam.ac.uk\nthisisjcykcd@gmail.com, brandenwang@tencent.com\nlemaoliu@gmail.com\nAbstract\nRecently, retrieval-augmented text generation\nattracted increasing attention of the compu-\ntational linguistics community.\nCompared\nwith conventional generation models, retrieval-\naugmented text generation has remarkable ad-\nvantages and particularly has achieved state-of-\nthe-art performance in many NLP tasks. This\npaper aims to conduct a survey about retrieval-\naugmented text generation. It ﬁrstly highlights\nthe generic paradigm of retrieval-augmented\ngeneration, and then it reviews notable ap-\nproaches according to different tasks including\ndialogue response generation, machine trans-\nlation, and other genera

In [ ]:
[str(docs.page_content()) for docs in vectordb.as_retriever().get_relevant_documents( eval_dataset[0]["question"])]

In [ ]:
eval_dataset.to_csv("/content/groundtruth_eval_dataset_new.csv")

## Final Evaluation Dataset

In [ ]:
def create_ragas_dataset(rag_pipeline, eval_dataset):
  rag_dataset = []
  for row in tqdm(eval_dataset):
    result = qa_chain({"query": row["question"]})
    rag_dataset.append(
        {"question" : row["question"],
         "answer" : result["result"],
         "contexts" : [str(docs.page_content) for docs in vectordb.as_retriever().get_relevant_documents( row["question"])],
         "ground_truth" : row["ground_truth"]
         }
    )
  rag_df = pd.DataFrame(rag_dataset)
  rag_eval_dataset = Dataset.from_pandas(rag_df)
  return rag_eval_dataset

In [ ]:
basic_qa_ragas_dataset = create_ragas_dataset(qa_chain, eval_dataset)

  0%|          | 0/10 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(

llama_print_timings:        load time =  196484.54 ms
llama_print_timings:      sample time =      45.80 ms /    79 runs   (    0.58 ms per token,  1725.00 tokens per second)
llama_print_timings: prompt eval time =  740205.29 ms /  1751 tokens (  422.73 ms per token,     2.37 tokens per second)
llama_print_timings:        eval time =   53087.82 ms /    78 runs   (  680.61 ms per token,     1.47 tokens per second)
llama_print_timings:       total time =  793631.19 ms /  1829 tokens
 10%|█         | 1/10 [13:13<1:59:03, 793.75s/it]Llama.generate: prefix-match hit

llama_print_timings:        load time =  196484.54 ms
llama_print_timings:      sample time =      91.98 ms /   162 runs   (    0.57 ms per token,  1761.29 

## Saving Dataset into CSV format

In [ ]:
basic_qa_ragas_dataset.to_csv("basic_qa_ragas_dataset_new.csv")

In [ ]:
from ragas import evaluate
from ragas.metrics import (
    answer_relevancy,
    faithfulness,
    context_recall,
    context_precision,
    context_relevancy,
    answer_correctness,
    answer_similarity
)

In [ ]:
def evaluate_ragas_dataset(ragas_dataset):
  result = evaluate(
    ragas_dataset,
    metrics=[
        context_precision,
        faithfulness,
        answer_relevancy,
        context_recall,
        context_relevancy,
        answer_correctness,
        answer_similarity
    ],
    raise_exceptions=False
  )
  return result

In [ ]:
basic_qa_result = evaluate_ragas_dataset(basic_qa_ragas_dataset)

In [ ]:
print(basic_qa_result)

In [ ]:
df = basic_qa_result.to_pandas()

In [ ]:
df.to_csv("basic_qa_result.csv")